In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 60
pd.options.display.max_rows = 500
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import datetime as dt

### Seteo Fechas

In [10]:
# Actualizar aca para extender en tiempo
start = '2003-01-31'
end = '2020-08-31'

## Forma actual

In [23]:

col_mon = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']

cpi = pd.read_csv('./../data/indice_precios.csv', index_col=0)
cpi.index = pd.date_range("1943-01", periods=len(cpi), freq = 'M')
# cpi = cpi.groupby(pd.TimeGrouper(freq='Q')).mean().loc['2003':'2019'][['index']]#.to_csv(...)
# cpi.set_index('index')
cpi.index.name = 'Q'
#
indice_precios = cpi

cpi = cpi.loc[start: end]   

cpi.columns = pd.MultiIndex.from_product([['Indice de Precios'], cpi.columns])


In [24]:
cpi.head()

,Indice de Precios
,index
Q,
2003-01-31,63.111588
2003-02-28,63.468811
2003-03-31,63.839314
2003-04-30,63.874584
2003-05-31,63.629604


In [25]:
# cpi = pd.read_csv('./../data/indice_precios.csv')
# cpi.index = pd.date_range("1943-01", periods=len(cpi), freq = 'M')
# cpi = cpi.groupby(pd.TimeGrouper(freq='Q')).mean().loc[str(startyr):str(endyr)][['index']]#.to_csv(...)
# cpi.set_index('index')
# cpi.index.name = 'Q'
# #
# indice_precios = cpi

# from pandas.tseries.offsets import MonthEnd

In [64]:
# CBAr = pd.read_csv('./../data/CBA_regional.csv')
CBAr = pd.read_csv('./../data/canasta-basica-alimentaria-regiones-del-pais.csv')
CBAr.index = pd.date_range(start = dt.datetime(2016,4,1),  periods=len(CBAr), freq='M')
CBAr = CBAr.drop(['indice_tiempo'], axis = 1)
CBAr = CBAr.rename(columns = {'gran_buenos_aires': 'Gran Buenos Aires', 'cuyo': 'Cuyo', 'noreste': 'Noreste', 
                      'noroeste': 'Noroeste', 'pampeana': 'Pampeana', 'patagonia': 'Patagónica'})

# CBTr = pd.read_csv('./../data/CBT_regional.csv')
CBTr = pd.read_csv('./../data/canasta-basica-total-regiones-del-pais.csv')
# CBTr.index = pd.date_range(start = dt.datetime(2016,4,1), end = dt.datetime(2019,6,30), freq='M')
# CBTr = CBTr.drop(['Ano4', 'Mes', 'TRIMESTRE'], axis = 1)
CBTr.index = pd.date_range(start = dt.datetime(2016,4,1),  periods=len(CBTr), freq='M')
CBTr = CBTr.drop(['indice_tiempo'], axis = 1)
CBTr = CBTr.rename(columns = {'gran_buenos_aires': 'Gran Buenos Aires', 'cuyo': 'Cuyo', 'noreste': 'Noreste', 
                      'noroeste': 'Noroeste', 'pampeana': 'Pampeana', 'patagonia': 'Patagónica'})

#.stack().reset_index()
CB = pd.concat([CBAr.stack(), CBTr.stack()], axis = 1)
CB.columns = ['CBA', 'CBT']; CB.index.names = ['Mes', 'Region']

In [65]:
# # 11747 (oct-nov 2019) se tiene que convertir en 
# 11747*(2283/1906) #para junio 2019
# # 11747*(1.231485) #para 2019Q2

In [66]:

# CB_i = pd.concat([CB.unstack(), cpi.iloc[:, 1]], axis = 1)
CB_i = pd.concat([CB.unstack(), cpi], axis = 1)
CB_defl = CB_i.iloc[:, :-1].div(CB_i.iloc[:, -1], 0)

# # Check how the fillna works...
# # fig, ax = plt.subplots(1, figsize = (7, 5))
# # CB_defl.fillna(CB_defl.mean()).plot(legend = False, ax = ax)
# # CB_defl.fillna(CB_defl.quantile(.25)).head(25).plot(legend = False, ax = ax, c = '.5')
# # CB_defl.fillna(CB_defl.quantile(.75)).head(25).plot(legend = False, ax = ax, c = '.5')

CB_defl = CB_defl.fillna(CB_defl.mean()).stack()
CB_defl_Q = CB_defl.groupby([pd.Grouper(level = 0,freq = 'Q'), pd.Grouper(level = 1)]).mean()
CB_defl_Q

# idx = CB_i.iloc[:, -1]['2019-11-30']
# CB_defl_Q = CB_defl_Q[['CBA', 'CBT']]*idx
CB_defl_Q = CB_defl_Q[['CBA', 'CBT']]*2283


CB_defl_Q.index.names = ['Q', 'Region']
CB_defl_Q = CB_defl_Q.reset_index()
CB_defl_Q[['CBA', 'CBT']] = CB_defl_Q[['CBA', 'CBT']]
CB_defl_Q.to_csv('./../data/CB_Reg_defl.csv', index = False)

## Listo

In [67]:
# Debug deflacion
# val_list = []
# for q in ['2018-09-30', '2018-06-30', '2018-03-31', '2017-12-31',
#        '2017-09-30', '2017-06-30', '2017-03-31', '2016-12-31',
#        '2016-09-30', '2016-06-30']:
#     df = pd.read_csv('./../data/RFReg_'+str(frac)+'ARG'+q+'.csv', 
#                      usecols = ['P47T'], nrows = 50000)
#     val_list += [df.median().values[0]]
# plt.plot(np.array(val_list))

In [68]:
CB_defl_Q.tail()

,Q,Region,CBA,CBT
421,2020-09-30,Gran Buenos Aires,4835.902013,11911.131444
422,2020-09-30,Noreste,4315.917733,9954.931062
423,2020-09-30,Noroeste,4204.770315,9613.770425
424,2020-09-30,Pampeana,4797.577940,11814.617967
425,2020-09-30,Patagónica,4985.734599,13940.959730


In [ ]:
## Rellena las canastas que faltan...
# CB_defl_Q.set_index(['Q', 'Region']).unstack()